In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError
from prophet import Prophet

# Import Data

In [56]:
mase = MeanAbsoluteScaledError(sp=4)
original = pd.read_csv('new_dataset/owid-covid-data-most-recent.csv')
df = pd.read_csv('multivariate.csv')
df['date'] = pd.to_datetime(df['date'])
df.rename(columns={'date':'ds', 'new_deaths':'y'}, inplace=True)
df = df[df['ds'] < '2024-01-14']
df['ds'] = pd.to_datetime(df['ds'])
train_size = int(len(df)*0.8)

# when total_cases is nan, set is to 0 
df['total_cases'] = df['total_cases'].fillna(0)

train, test = df[0:train_size], df[train_size:]

df.shape

(2020, 51)

In [11]:
# time events

# COVID time window
COVID = pd.DataFrame({
    'holiday': 'COVID',
    'ds': pd.to_datetime(['2020-03-15']),
    'lower_window': -15,
    'upper_window': 15,
})

# Super Bowl time window
superbowl = pd.DataFrame({
    'holiday': 'superbowl',
    'ds': pd.to_datetime(['2020-02-02', '2021-02-07','2023-2-12']),
    'lower_window': -7,
    'upper_window': 1,
})

# vaccine approval time window
vaccine = pd.DataFrame({
    'holiday': 'vaccine',
    'ds': pd.to_datetime(['2020-12-11']),
    'lower_window': -1,
    'upper_window': 31,
})
# Combine all events
events = pd.concat((COVID, superbowl, vaccine))

# Take a look at the events data
events

,holiday,ds,lower_window,upper_window
0,COVID,2020-03-15,-15,15
0,superbowl,2020-02-02,-7,1
1,superbowl,2021-02-07,-7,1
2,superbowl,2023-02-12,-7,1
0,vaccine,2020-12-11,-1,31


# Model Building

In [58]:

# adding holidays
model = Prophet(holidays=events,yearly_seasonality=True)

# add built_in country specific holidays 
model.add_country_holidays(country_name='US')

#adding regressors
for col in train.columns:
    if col not in ['ds','y']:
        model.add_regressor(col, standardize=False)
model= model.fit(train.reset_index(drop=True))
# Create the time range for the forecast
future_multivariate = model.make_future_dataframe(periods=41)

# Append the regressor values
future_multivariate = pd.merge(future_multivariate, df, on='ds', how='inner')

# Fill the missing values with the previous value
future_multivariate = future_multivariate.fillna(method='ffill')

# Check the data
future_multivariate.tail(10)




ValueError: Found NaN in column 'total_cases_per_million'